In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
RIO_geo = pd.read_csv('../Datasets/Geography_dataset/RIO_geo.csv', index_col = False)

In [3]:
RIO_geo

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39
0,Rio Tinto PLC (RIO LN) - By Geography,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,In Millions of USD except Per Share,NaN,FY 1998,NaN,FY 2004,NaN,FY 2005,NaN,FY 2006,NaN,...,FY 2017,NaN,FY 2018,NaN,FY 2019,NaN,FY 2020,NaN,FY 2021,NaN
3,12 Months Ending,NaN,12/31/1998,NaN,12/31/2004,NaN,12/31/2005,NaN,12/31/2006,NaN,...,12/31/2017,NaN,12/31/2018,NaN,12/31/2019,NaN,12/31/2020,NaN,12/31/2021,NaN
4,Revenue,NaN,"7,112.0",100.0%,"12,954.0",100.0%,"19,033.0",100.0%,"22,465.0",100.0%,...,"40,030.0",100.0%,"40,522.0",100.0%,"43,165.0",100.0%,"44,611.0",100.0%,"63,495.0",100.0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Australia,NaN,"8,735.00",25.09%,?,NaN,?,NaN,?,NaN,...,?,NaN,?,NaN,?,NaN,?,NaN,?,NaN
86,Rest of the World,NaN,"1,114.00",3.20%,?,NaN,?,NaN,?,NaN,...,?,NaN,?,NaN,?,NaN,?,NaN,?,NaN
87,Europe,NaN,"3,596.00",10.33%,?,NaN,?,NaN,?,NaN,...,?,NaN,?,NaN,?,NaN,?,NaN,?,NaN
88,North America,NaN,"7,782.00",22.36%,?,NaN,?,NaN,?,NaN,...,?,NaN,?,NaN,?,NaN,?,NaN,?,NaN


In [4]:
def Geo_Wrangler(df):
    df.columns = df.loc[3].values
    df['12 Months Ending'] = df['12 Months Ending'].str.strip()
    df = df.loc[:, df.columns.notna()]
    df = df.drop([0, 1, 2, 3, len(df) - 1], axis = 0)
    df = df.reset_index(drop = True)
    
    rev_idx = df['12 Months Ending'][df['12 Months Ending'] == 'Revenue'].index[0]
    ass_sale_idx = df['12 Months Ending'][df['12 Months Ending'] == 'Assets Held for Sale'].index[0]
    
    return df[rev_idx : ass_sale_idx]

In [5]:
RIO_geo = Geo_Wrangler(RIO_geo)

In [6]:
RIO_geo

,12 Months Ending,12/31/1998,12/31/2004,12/31/2005,12/31/2006,12/31/2007,12/31/2008,12/31/2009,12/31/2010,12/31/2011,12/31/2012,12/31/2013,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,12/31/2019,12/31/2020,12/31/2021
0,Revenue,"7,112.0","12,954.0","19,033.0","22,465.0","29,700.0","54,264.0","41,825.0","55,171.0","60,537.0","50,967.0","51,171.0","47,664.0","34,829.0","33,781.0","40,030.0","40,522.0","43,165.0","44,611.0","63,495.0"
1,China,?,"1,389.0","2,932.0","3,648.0","5,342.0","10,934.0","10,691.0","16,581.0","20,155.0","17,948.0","19,331.0","19,101.0","14,558.0","14,405.0","17,706.0","18,061.0","22,135.0","25,940.0","36,308.0"
2,North America,"2,582.0","3,314.0","4,235.0","5,358.0","7,262.0","12,984.0","10,190.0","10,900.0","10,945.0","8,908.0","8,418.0","7,860.0","6,780.0","5,861.0","6,827.0","7,618.0","7,603.0","6,156.0","9,689.0"
3,United States,?,?,?,?,?,?,?,"8,975.0","9,019.0","7,085.0","7,142.0","6,439.0","5,292.0","4,762.0","5,716.0","6,278.0","6,125.0","4,867.0","8,012.0"
4,Canada,?,?,?,?,?,?,?,"1,925.0","1,926.0","1,823.0","1,276.0","1,421.0","1,488.0","1,099.0","1,111.0","1,340.0","1,478.0","1,289.0","1,677.0"
5,Other Asia,?,"1,676.0","2,366.0","2,691.0","3,238.0","6,453.0","5,822.0","8,603.0","10,410.0","8,464.0","8,313.0","7,913.0","4,695.0","5,011.0","5,108.0","4,665.0","4,558.0","4,536.0","5,985.0"
6,Japan,?,"2,319.0","3,620.0","4,402.0","5,012.0","8,825.0","5,921.0","9,410.0","10,892.0","8,787.0","8,770.0","7,719.0","3,907.0","3,681.0","4,701.0","3,873.0","3,855.0","3,354.0","5,012.0"
7,Europe,636.0,"2,607.0","3,968.0","3,929.0","6,027.0","12,015.0","6,337.0","8,340.0","7,549.0","6,380.0","5,552.0","4,407.0","2,783.0","2,621.0","3,015.0","3,706.0","2,610.0","2,623.0","3,271.0"
8,Other Countries,-571.0,479.0,576.0,"1,025.0","1,048.0","3,005.0","2,541.0","2,882.0","3,102.0","3,012.0","2,460.0","1,446.0",900.0,"1,135.0","1,514.0","1,493.0","1,419.0","1,015.0","1,865.0"
9,Australia,"3,080.0",?,?,?,?,?,?,?,?,"1,420.0","1,114.0","1,114.0",867.0,715.0,710.0,720.0,737.0,745.0,"1,122.0"


In [7]:
RIO_geo['12 Months Ending'].values

array(['Revenue', 'China', 'North America', 'United States', 'Canada',
       'Other Asia', 'Japan', 'Europe', 'Other Countries', 'Australia',
       'United Kingdom'], dtype=object)

In [8]:
# rev_idx = RIO_geo['12 Months Ending'][RIO_geo['12 Months Ending'] == 'Revenue'].index[0]
# ass_sale_idx = RIO_geo['12 Months Ending'][RIO_geo['12 Months Ending'] == 'Assets Held for Sale'].index[0]
# RIO_geo[0:11]

In [9]:
# print(rev_idx)
# print(ass_sale_idx)